# Paper revision

## Load annotation feature

(As we have multiple gene isoforms, we will consider only the .1 one)

In [1]:
import pandas as pd
import yaml
from IPython.display import clear_output, display

In [2]:
annotation = pd.read_csv(
    "~/databases/vitis/V2.1.gff3",
    sep="\t",
    header=None
    )

# annotation.head(n=20)

## List of features to test

In [3]:
feature_size = {
    "cds": 0,
    "downstream": 0,
    "exon_length": 0,
    "exon_number": 0,
    "gene_length": 0,
    "gene_number": 0,
    "intergenic": 0,
    "intron": 0,
    "splicing_site": 0,
    "start_stop": 0,
    "upstream": 0,
    "utr_3": 0,
    "utr_5": 0
    }

In [4]:
# Remember, we will only consider the 1st isoform!
for index, row in annotation[(annotation[2] == "CDS") & (annotation[8].str.contains(".1.cds"))].iterrows():
    feature_size["cds"] += row[4] - row[3]

for index, row in annotation[(annotation[2] == "exon") & (annotation[8].str.contains(".1.exon"))].iterrows():
    feature_size["exon_length"] += row[4] - row[3]

# Genes does not consider isoforms
for index, row in annotation[(annotation[2] == "gene")].iterrows():
    feature_size["gene_length"] += row[4] - row[3]

for index, row in annotation[(annotation[2] == "three_prime_utr") & (annotation[8].str.contains(".1.utr3p"))].iterrows():
    feature_size["utr_3"] += row[4] - row[3]

for index, row in annotation[(annotation[2] == "five_prime_utr") & (annotation[8].str.contains(".1.utr5p"))].iterrows():
    feature_size["utr_5"] += row[4] - row[3]

# print(yaml.dump(feature_size))

In [5]:
# Remember, we will only consider the 1st isoform!
feature_size["exon_number"] = len(annotation[(annotation[2] == "exon") & (annotation[8].str.contains(".1.exon"))])

# Genes does not consider isoform
feature_size["gene_number"] = len(annotation[(annotation[2] == "gene")])

# print(yaml.dump(feature_size))

In [6]:
feature_size["intron"] = feature_size["gene_length"] - feature_size["exon_length"]

junctions = (feature_size["exon_number"] - feature_size["gene_number"]) * 2
feature_size["splicing_site"] = junctions * 2

feature_size["start_stop"] = feature_size["gene_number"] * 3

# print(yaml.dump(feature_size))

In [7]:
# Subset genes
genes = annotation[annotation[2] == "gene"].copy()
genes["upstream_start"] = None
genes["upstream_end"] = None
genes["downstream_start"] = None
genes["downstream_end"] = None
genes["domain_start"] = None
genes["domain_end"] = None

# Loop over genes to infer upstream and downstream
for index, row in genes.iterrows():
    if row[6] == "+":
        # Upstream is before and downstream is after
        gene_start = int(row[3])
        gene_end = int(row[4])
        
        gene_upstream = gene_start - 5000
        gene_downstream = gene_end + 5000
        
    else:
        # Upstream is after and downstream is before
        gene_start = int(row[4])
        gene_end = int(row[3])
        
        gene_upstream = gene_start + 5000
        gene_downstream = gene_start - 5000
    
    genes.at[index, "upstream_start"] = min(gene_upstream, gene_start)
    genes.at[index, "upstream_end"] = max(gene_upstream, gene_start)
    genes.at[index, "downstream_start"] = min(gene_downstream, gene_end)
    genes.at[index, "downstream_end"] = max(gene_downstream, gene_end)
    genes.at[index, "domain_start"] = min(gene_upstream, gene_downstream)
    genes.at[index, "domain_end"] = max(gene_upstream, gene_downstream)

# genes.head()

In [8]:
idx_max = genes.index.max()

# Loop over genes to infer Up/Down-stream
for index, row in genes.iterrows():
    clear_output(wait=True)
    print(f"{index}/{idx_max} ({round(index/idx_max*100,1)}%)")
    # If no other gene touch in this gene's domain we may considere the entire upstream and downstream regions without problems
    genes_touching = genes[(genes[0] == row[0])
                           & (
                               (
                                   (genes["domain_start"] >= row["domain_start"])
                                   & (genes["domain_start"] <= row["domain_end"])
                               )
                               | (
                                   (genes["domain_end"] >= row["domain_start"])
                                   & (genes["domain_end"] <= row["domain_end"])
                               )
                           )
                           & (genes[8] != row[8])
                          ].copy()
    
    if len(genes_touching.index) == 0:
        feature_size["upstream"] += 5000
        feature_size["downstream"] += 5000
        continue
    
    
    
    ####################
    ### UPSTREAM ONY ###
    ####################
    
    # Verify if upstream touch other genes
    genes_touching_upstream = genes_touching[(
        ((genes_touching["domain_start"] >= row["upstream_start"]) & (genes_touching["domain_start"] <= row["upstream_end"]))
        | ((genes_touching["domain_end"] >= row["upstream_start"]) & (genes_touching["domain_end"] <= row["upstream_end"]))
        )].copy()
    
    if len(genes_touching_upstream.index) == 0:
        feature_size["upstream"] += 5000
        # We can't continue right here, beacause we still need to evaluate downstream
    else:
        row_upstream_start = row["upstream_start"]
        row_upstream_end = row["upstream_end"]
        
        # Remove portion associated to genes
        genes_touching_upstream_gene = genes_touching_upstream[(
            ((genes_touching_upstream[3] >= row["upstream_start"]) & (genes_touching_upstream[3] <= row["upstream_end"]))
            | ((genes_touching_upstream[4] >= row["upstream_start"]) & (genes_touching_upstream[4] <= row["upstream_end"]))
            )].copy()
        
        for second_index, second_row in genes_touching_upstream_gene.iterrows():
            if row[6] == "+":
                row_upstream_start = max(second_row[4], row_upstream_start)
            else:
                row_upstream_end = min(second_row[3], row_upstream_end)
        
        upstream_count = row_upstream_end - row_upstream_start
        if upstream_count < 0:
            upstream_count = 0
        
        else:
            # Remove portion associated with other upstreams
            # We will count those upstreams as half and so, when both genes are analysed, each place will sum 1!
            
            genes_touching_upstream_upstream = genes_touching_upstream[(
                ((genes_touching_upstream["upstream_start"] >= row_upstream_start) & (genes_touching_upstream["upstream_start"] <= row_upstream_end))
                | ((genes_touching_upstream["upstream_end"] >= row_upstream_start) & (genes_touching_upstream["upstream_end"] <= row_upstream_end))
                )].copy()
            
            gene_upstream_range = set(range(row_upstream_start, row_upstream_end))
            for second_index, second_row in genes_touching_upstream_upstream.iterrows():
                second_upstream_range = set(range(second_row["upstream_start"], second_row["upstream_end"]))
                upstream_intercection = len(gene_upstream_range.intersection(second_upstream_range))
                upstream_count = upstream_count - upstream_intercection/2
        
        feature_size["upstream"] += upstream_count
        
    
    ######################
    ### DOWNSTREAM ONY ###
    ######################
    
    # Verify if downstream touch other genes
    genes_touching_downstream = genes_touching[(
        ((genes_touching["domain_start"] >= row["downstream_start"]) & (genes_touching["domain_start"] <= row["downstream_end"]))
        | ((genes_touching["domain_end"] >= row["downstream_start"]) & (genes_touching["domain_end"] <= row["downstream_end"]))
        )].copy()
    
    if len(genes_touching_downstream.index) == 0:
        feature_size["downstream"] += 5000
        # We could continue right now, we we will not to keep consistency to previous code
    else:
        row_downstream_start = row["downstream_start"]
        row_downstream_end = row["downstream_end"]
        
        # Remove portion associated to genes
        genes_touching_downstream_gene = genes_touching_downstream[(
            ((genes_touching_downstream[3] >= row["downstream_start"]) & (genes_touching_downstream[3] <= row["downstream_end"]))
            | ((genes_touching_downstream[4] >= row["downstream_start"]) & (genes_touching_downstream[4] <= row["downstream_end"]))
            )].copy()
        
        for second_index, second_row in genes_touching_downstream_gene.iterrows():
            if row[6] == "+":
                row_downstream_end = min(second_row[3], row_downstream_end)
            else:
                row_downstream_start = max(second_row[4], row_downstream_start)
        
        downstream_count = row_downstream_end - row_downstream_start
        if downstream_count < 0:
            downstream_count = 0
        
        else:
            
            # Remove portion associated with upstreams
            # Upstream has priority
            genes_touching_downstream_upstream = genes_touching_downstream[(
                ((genes_touching_downstream["upstream_start"] >= row_downstream_start) & (genes_touching_downstream["upstream_start"] <= row_downstream_end))
                | ((genes_touching_downstream["upstream_end"] >= row_downstream_start) & (genes_touching_downstream["upstream_end"] <= row_downstream_end))
                )].copy()

            for second_index, second_row in genes_touching_downstream_upstream.iterrows():
                if row[6] == "+":
                    row_downstream_end = min(second_row["upstream_start"], row_downstream_end)
                else:
                    row_downstream_start = max(second_row["upstream_end"], row_downstream_start)

            downstream_count = row_downstream_end - row_downstream_start
            if downstream_count < 0:
                downstream_count = 0

            else:

                # Evaluate other downstreams!            
                # We will count those upstreams as half and so, when both genes are analysed, each place will sum 1!

                genes_touching_downstream_downstream = genes_touching_downstream[(
                    ((genes_touching_downstream["downstream_start"] >= row_downstream_start) & (genes_touching_downstream["downstream_start"] <= row_downstream_end))
                    | ((genes_touching_downstream["downstream_end"] >= row_downstream_start) & (genes_touching_downstream["downstream_end"] <= row_downstream_end))
                    )].copy()

                gene_downstream_range = set(range(row_downstream_start, row_downstream_end))
                for second_index, second_row in genes_touching_downstream_downstream.iterrows():
                    second_downstream_range = set(range(second_row["downstream_start"], second_row["downstream_end"]))
                    downstream_intercection = len(gene_downstream_range.intersection(second_downstream_range))
                    downstream_count = downstream_count - downstream_intercection/2

        feature_size["downstream"] += downstream_count
        
# print(yaml.dump(feature_size))

820939/820939 (100.0%)


Let's calculate intergenics!

In [9]:
chr_len = pd.read_csv("chr_len.tsv",
                      sep="\t",
                      header=0,
                      index_col=0
                     )

# chr_len.head(n=2)

In [10]:
total_genome_length = chr_len["Length"].sum()
feature_size["intergenic"] = int(total_genome_length - (feature_size["upstream"] + feature_size["downstream"] + feature_size["gene_length"]))
# feature_size["intergenic"]

In [11]:
with open('feature_size.yaml', 'w') as output_file:
    yaml.dump(feature_size, output_file)

# print(yaml.dump(feature_size))